In [2]:
import pandas as pd

dataset = pd.read_csv('housing.csv')

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [6]:
print(dataset.shape)
dataset[:5]

(506, 13)


,medv,crim,zn,indus,chas,nox,age,rm,dis,rad,tax,ptratio,lstat
0,24.0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,4.98
1,21.6,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,9.14
2,34.7,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,4.03
3,33.4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,2.94
4,36.2,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,5.33


In [7]:
# Move 'medv' column to front
dataset = pd.concat([dataset['medv'], dataset.drop(['medv'], axis=1)], axis=1)

In [8]:
from sklearn.model_selection import train_test_split
training_dataset, validation_dataset = train_test_split(dataset, test_size=0.1)

print(training_dataset.shape)
print(validation_dataset.shape)

(455, 13)
(51, 13)


In [9]:
training_dataset.to_csv('training_dataset.csv', index=False, header=False)
validation_dataset.to_csv('validation_dataset.csv', index=False, header=False)

In [10]:
import sagemaker

print(sagemaker.__version__)

sess = sagemaker.Session()
bucket = sess.default_bucket()

prefix = 'boston-housing'
training_data_path = sess.upload_data(path='training_dataset.csv', key_prefix=prefix + '/input/training')
validation_data_path = sess.upload_data(path='validation_dataset.csv', key_prefix=prefix + '/input/validation')

print(training_data_path)
print(validation_data_path)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
2.199.0
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
s3://sagemaker-us-east-1-289456105026/boston-housing/input/training/training_dataset.csv
s3://sagemaker-us-east-1-289456105026/boston-housing/input/validation/validation_dataset.csv


In [11]:
import boto3
from sagemaker import image_uris

region = boto3.Session().region_name

# Use XGBoost container
container = image_uris.retrieve("xgboost", region, version="1.2-1")
print(container)


from sagemaker.estimator import Estimator

role = sagemaker.get_execution_role() 

# Use XGBoost container
container = image_uris.retrieve("xgboost", region, version="1.2-1")

xgboost_estimator = Estimator(container,
    role=role, 
    instance_count=1,
    instance_type='ml.m5.large',
    output_path='s3://{}/{}/output'.format(bucket, prefix)
)

# Set XGBoost hyperparameters
xgboost_estimator.set_hyperparameters(
    objective='reg:squarederror',
    num_round=100,
    max_depth=5,
    eta=0.2
)


683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.2-1
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [12]:
training_data_channel   = sagemaker.TrainingInput(s3_data=training_data_path, content_type='text/csv')
validation_data_channel = sagemaker.TrainingInput(s3_data=validation_data_path, content_type='text/csv')

ll_data = {'train': training_data_channel, 'validation': validation_data_channel}

In [14]:
xgboost_estimator.fit(ll_data)

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-12-06-21-58-41-725


2023-12-06 21:58:41 Starting - Starting the training job...
2023-12-06 21:58:58 Starting - Preparing the instances for training.........
2023-12-06 22:00:19 Downloading - Downloading input data......
2023-12-06 22:01:14 Training - Downloading the training image...
2023-12-06 22:02:05 Uploading - Uploading generated training model[2023-12-06 22:01:57.971 ip-10-0-71-196.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value reg:squarederror to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV in

In [15]:
%%bash -s "$xgboost_estimator.output_path"
aws s3 ls --recursive $1

2023-12-06 22:01:59          0 boston-housing/output/sagemaker-xgboost-2023-12-06-21-58-41-725/debug-output/claim.smd
2023-12-06 22:01:59       6276 boston-housing/output/sagemaker-xgboost-2023-12-06-21-58-41-725/debug-output/collections/000000000/worker_0_collections.json
2023-12-06 22:01:59        218 boston-housing/output/sagemaker-xgboost-2023-12-06-21-58-41-725/debug-output/events/000000000000/000000000000_worker_0.tfevents
2023-12-06 22:01:59        224 boston-housing/output/sagemaker-xgboost-2023-12-06-21-58-41-725/debug-output/events/000000000010/000000000010_worker_0.tfevents
2023-12-06 22:01:59        224 boston-housing/output/sagemaker-xgboost-2023-12-06-21-58-41-725/debug-output/events/000000000020/000000000020_worker_0.tfevents
2023-12-06 22:01:59        224 boston-housing/output/sagemaker-xgboost-2023-12-06-21-58-41-725/debug-output/events/000000000030/000000000030_worker_0.tfevents
2023-12-06 22:01:59        224 boston-housing/output/sagemaker-xgboost-2023-12-06-21-58-41

In [16]:
from time import strftime, gmtime
timestamp = strftime('%d-%H-%M-%S', gmtime())

endpoint_name = 'xgboost-demo-'+timestamp
print(endpoint_name)

xgboost-demo-06-22-07-03


In [17]:
xgboost_predictor = xgboost_estimator.deploy(endpoint_name=endpoint_name, 
                        initial_instance_count=1, 
                        instance_type='ml.t2.medium')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2023-12-06-22-07-56-685
INFO:sagemaker:Creating endpoint-config with name xgboost-demo-06-22-07-03
INFO:sagemaker:Creating endpoint with name xgboost-demo-06-22-07-03


--------!

In [18]:
test_sample = '0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,4.98'

In [19]:
#ll_predictor.content_type = 'text/csv'
xgboost_predictor.serializer = sagemaker.serializers.CSVSerializer()
xgboost_predictor.deserializer = sagemaker.deserializers.CSVDeserializer()

response = xgboost_predictor.predict(test_sample)
print(response)

[['24.024606704711914']]


In [21]:
test_samples = ['0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,4.98',
                '0.02731,0.00,7.070,0,0.4690,6.4210,78.90,4.9671,2,242.0,17.80,9.14']

response = xgboost_predictor.predict(test_samples)
print(response)
print(xgboost_predictor.endpoint_name)

[['24.024606704711914', '21.335205078125']]
xgboost-demo-06-22-07-03


In [22]:
runtime = boto3.Session().client(service_name='runtime.sagemaker') 

response = runtime.invoke_endpoint(EndpointName=endpoint_name, 
                                  ContentType='text/csv', 
                                  Body=test_sample)

print(response['Body'].read())

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


b'24.024606704711914'


In [ ]:
xgboost_predictor.delete_endpoint()